In [1]:
import sys
import os
import math
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from math import sqrt
from sklearn.metrics import confusion_matrix

if not sys.warnoptions:
    warnings.simplefilter("ignore")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
sns.set_style('white')

import plotly.graph_objects as go
from datetime import datetime

In [2]:
# read data
year = 2018
ticker = 'IFM'

col_names=['TIME', 'X', 'Y', 'Z'] 

df_2018 = pd.read_csv(os.path.join(f'../data/index/{year}/{ticker}.csv'), index_col=0, encoding='gbk')
df_2018.columns = ['code', 'time', 'open', 'high', 'low', 'close', 'volume', 'turnover', 'open_interest']

df_2018.rename_axis("type", axis='index', inplace=True)

year = 2019

df_2019 = pd.read_csv(os.path.join(f'../data/index/{year}/{ticker}.csv'), index_col=0, encoding='gbk')
df_2019.columns = ['code', 'time', 'open', 'high', 'low', 'close', 'volume', 'turnover', 'open_interest']

df_2019.rename_axis("type", axis='index', inplace=True)

year = 2020

df_2020 = pd.read_csv(os.path.join(f'../data/index/{year}/{ticker}.csv'), index_col=0, encoding='gbk')
df_2020.columns = ['code', 'time', 'open', 'high', 'low', 'close', 'volume', 'turnover', 'open_interest']

df_2020.rename_axis("type", axis='index', inplace=True)


df = pd.concat([df_2018, df_2019, df_2020])
df['time'] = pd.to_datetime(df['time'])

df.reset_index(inplace=True)
df['start'] = df['time'] - pd.Timedelta(minutes=1)
df['end'] = df['time']

df['date'] = df['time'].map(lambda x: x.year)*10000 + df['time'].map(lambda x: x.month)*100 + df['time'].map(lambda x: x.day)
# df = df.iloc[144000:146400, :] # Change this line to change the selection of data

df

,type,code,time,open,high,low,close,volume,turnover,open_interest,start,end,date
0,SF,IF1801,2018-01-02 09:31:00,4051.6,4060.4,4051.6,4060.0,398.0,4.844222e+08,24365,2018-01-02 09:30:00,2018-01-02 09:31:00,20180102
1,SF,IF1801,2018-01-02 09:32:00,4060.0,4062.2,4058.6,4060.0,212.0,2.582576e+08,24241,2018-01-02 09:31:00,2018-01-02 09:32:00,20180102
2,SF,IF1801,2018-01-02 09:33:00,4059.8,4065.8,4059.8,4062.8,179.0,2.181614e+08,24118,2018-01-02 09:32:00,2018-01-02 09:33:00,20180102
3,SF,IF1801,2018-01-02 09:34:00,4062.6,4063.0,4060.0,4062.6,117.0,1.425521e+08,24049,2018-01-02 09:33:00,2018-01-02 09:34:00,20180102
4,SF,IF1801,2018-01-02 09:35:00,4062.8,4070.8,4062.4,4070.0,251.0,3.062792e+08,23926,2018-01-02 09:34:00,2018-01-02 09:35:00,20180102
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175195,SF,IF2101,2020-12-31 14:56:00,5217.8,5218.4,5216.4,5217.8,468.0,7.325071e+08,89496,2020-12-31 14:55:00,2020-12-31 14:56:00,20201231
175196,SF,IF2101,2020-12-31 14:57:00,5217.8,5219.8,5216.6,5219.0,502.0,7.858570e+08,89727,2020-12-31 14:56:00,2020-12-31 14:57:00,20201231
175197,SF,IF2101,2020-12-31 14:58:00,5219.0,5219.0,5215.0,5218.2,704.0,1.101811e+09,90073,2020-12-31 14:57:00,2020-12-31 14:58:00,20201231
175198,SF,IF2101,2020-12-31 14:59:00,5218.2,5221.6,5216.8,5220.4,610.0,9.551740e+08,90469,2020-12-31 14:58:00,2020-12-31 14:59:00,20201231


In [3]:
from cta.kline import Kline
from cta.kline_general import *

SyntaxError: invalid syntax (_fractal.py, line 2)

In [ ]:
timespan = df.shape[0]
frequency = 30

df_kline = Kline_General(df, 1)

In [ ]:
# plot 30 min k-line
df_kline30 = df_kline.construct_backward_klines(ckfreq=frequency, numkrows=(timespan // frequency))
# df_kline30.plot_k()
temp = df_kline30.ma_trend_filter(ma=5).iloc[::-1]
temp.iloc[0:20, ]

In [ ]:
frequency = 120
# plot 120 min k-line
# df_kline120 = df_kline.construct_backward_klines(ckfreq=frequency, numkrows=(timespan // frequency))
# df_kline120.plot_k()

计算高点和低点的 Hull Moving Average, 可以调整 hma_span

In [ ]:
# reverse the dataframe since the k-line lib reversed it
df_kline30r = df_kline30.iloc[::-1].copy()
df_kline30r.reset_index(inplace=True)
hma_span = 5 # period for HMA 

df_kline30r['window_t_ewm_low'] = df_kline30r['low'].ewm(span=hma_span).mean()
df_kline30r['window_half_ewm_low'] = df_kline30r['low'].ewm(span=hma_span/2).mean()
df_kline30r['diff_low']=df_kline30r['window_half_ewm_low'] * 2 - df_kline30r['window_t_ewm_low']
df_kline30r['hma_low']=df_kline30r['diff_low'].ewm(span=int(math.sqrt(hma_span))).mean()

df_kline30r['window_t_ewm_high'] = df_kline30r['high'].ewm(span=hma_span).mean()
df_kline30r['window_half_ewm_high'] = df_kline30r['high'].ewm(span=hma_span/2).mean()
df_kline30r['diff_high']=df_kline30r['window_half_ewm_high'] * 2 - df_kline30r['window_t_ewm_high']
df_kline30r['hma_high']=df_kline30r['diff_high'].ewm(span=int(math.sqrt(hma_span))).mean()

df_kline30r['window_t_ewm_close'] = df_kline30r['close'].ewm(span=hma_span).mean()
df_kline30r['window_half_ewm_close'] = df_kline30r['close'].ewm(span=hma_span/2).mean()
df_kline30r['diff_close']=df_kline30r['window_half_ewm_close'] * 2 - df_kline30r['window_t_ewm_close']
df_kline30r['hma_close']=df_kline30r['diff_close'].ewm(span=int(math.sqrt(hma_span))).mean()

intermediate_col = ['index', 'window_t_ewm_low', 'window_half_ewm_low', 'diff_low', 
                    'window_t_ewm_high', 'window_half_ewm_high', 'diff_high', 
                   'window_t_ewm_close', 'window_half_ewm_close', 'diff_close']

df_kline30r.drop(columns=intermediate_col, inplace = True)

df_kline30r[['hma_close', 'hma_high', 'hma_low']].plot(grid=False, figsize=(12,4))
sns.despine()

通过 HMA 的斜率计算趋势，只有当高线和低线趋势一致时才判断趋势存在

In [ ]:
df_kline30r["trend_low"] = (df_kline30r["hma_low"] > df_kline30r["hma_low"].shift(1))
df_kline30r["trend_high"] = (df_kline30r["hma_high"] > df_kline30r["hma_high"].shift(1))

df_kline30r["slope_low"] = (df_kline30r["hma_low"] - df_kline30r["hma_low"].shift(1))
df_kline30r["slope_high"] = (df_kline30r["hma_high"] - df_kline30r["hma_high"].shift(1))
df_kline30r["slope_close"] = (df_kline30r["hma_close"] - df_kline30r["hma_close"].shift(1))

df_kline30r['slope_low'] = df_kline30r['slope_low'].fillna(0)
df_kline30r['slope_high'] = df_kline30r['slope_high'].fillna(0)
df_kline30r["slope_close"] = df_kline30r['slope_close'].fillna(0)

df_kline30r.replace({'trend_low': {True: 1, False: -1}}, inplace=True)
df_kline30r.replace({'trend_high': {True: 1, False: -1}}, inplace=True)

df_kline30r["trend_ma"] = df_kline30r["trend_low"] + df_kline30r["trend_high"]

df_kline30r.replace({'trend_ma': {2: 1, -2: -1}}, inplace=True)

df_kline30r

计算 Average True Range，可以更改 atr_span

In [ ]:
atr_span = 10
def calculate_average_true_range(df, atr_span):
    df['TR1'] = abs(df['high'] - df['low'])
    df['TR2'] = abs(df['high'] - df['close'].shift())
    df['TR3'] = abs(df['low'] - df['close'].shift())
    df['true_range'] = df[['TR1', 'TR2', 'TR3']].max(axis=1)
    df['atr'] = df['true_range'].ewm(span = atr_span).mean()
    
    df_kline30r.drop(columns=['TR1', 'TR2', 'TR3'], inplace = True)
    
    return df

df_kline30r = calculate_average_true_range(df_kline30r, 10)

通过 ATR 判断趋势 仅当高线和低线的均线变化幅度均超过 ATR * M 时判断趋势存在  
可以调整 M 的取值

In [ ]:
atr_m = 0.1
df_kline30r["trend_atr_up"] = ((df_kline30r["slope_high"] > df_kline30r["atr"] * atr_m) & 
                               (df_kline30r["slope_low"] > df_kline30r["atr"] * atr_m) & 
                              (df_kline30r["slope_close"] > df_kline30r["atr"] * atr_m) )
df_kline30r["trend_atr_down"] = ((df_kline30r["slope_high"] < df_kline30r["atr"] * -atr_m) & 
                               (df_kline30r["slope_low"] < df_kline30r["atr"] * -atr_m) & 
                                (df_kline30r["slope_close"] < df_kline30r["atr"] * atr_m))

df_kline30r.replace({'trend_atr_up': {True: 1, False: 0}}, inplace=True)
df_kline30r.replace({'trend_atr_down': {True: -1, False: 0}}, inplace=True)

df_kline30r["trend_atr"] = df_kline30r["trend_atr_up"] + df_kline30r["trend_atr_down"]
df_kline30r["trend"] = df_kline30r["trend_atr"] + df_kline30r["trend_ma"]
df_kline30r.replace({'trend': {2: 1, 1: 0, -1: 0, -2: -1}}, inplace=True)

df_kline30r["trend"] = (df_kline30r["trend"] * 2 + df_kline30r["trend"].shift()) / 3
df_kline30r['trend'] = df_kline30r['trend'].fillna(0)
df_kline30r["trend"] = df_kline30r["trend"].round()

df_kline30r.iloc[0:20, ]

在30分钟K线上增加 Trend

In [ ]:
# Inverse here because we inversed before
df_kline30["high_ma5"] = df_kline30r['hma_high'].values[::-1]
df_kline30["low_ma5"] = df_kline30r['hma_low'].values[::-1]
df_kline30["trend5"] = df_kline30r['trend'].values[::-1]

# df_kline30.plot_k(plot_ma=True, ma=5)

In [ ]:
df_kline30

In [ ]:
df_kline30['date'] = df_kline30.end.dt.date.astype(str)
df_kline30['start_time'] = df_kline30.start.dt.time.astype(str)
df_kline30['end_time'] = df_kline30.end.dt.time.astype(str)

In [ ]:
df_kline30.head(10)

In [ ]:
daily_close = df_kline30[df_kline30.end_time=='15:00:00']
daily_open = df_kline30[df_kline30.start_time=='09:30:00']

In [ ]:
daily_close['next_day_open'] = daily_open.shift(1).open.tolist()

In [ ]:
daily_close = daily_close.dropna()

In [ ]:
daily_close['night_ret'] = (daily_close.next_day_open - daily_close.close)

In [ ]:
daily_close['night_ret'].describe()

In [ ]:
daily_close[daily_close.trend5==1]['night_ret'].describe()

In [ ]:
daily_close[daily_close.trend5==0]['night_ret'].describe()

In [ ]:
daily_close[daily_close.trend5==-1]['night_ret'].describe()